# Test 04 — BoardSDK buzzer + RGB

This probes BoardSDK methods and attempts:
- buzzer beep
- RGB set (unknown signature -> we probe and try common patterns)


In [ ]:
# --- Bootstrap: add repo root + common/lib to sys.path ---
from pathlib import Path
import sys

def add_repo_to_path():
    here = Path.cwd().resolve()
    for p in [here] + list(here.parents):
        if (p / 'lessons').is_dir() and (p / 'common').is_dir():
            repo_root = p
            common_lib = p / 'common' / 'lib'
            if str(repo_root) not in sys.path:
                sys.path.insert(0, str(repo_root))
            if str(common_lib) not in sys.path:
                sys.path.insert(0, str(common_lib))
            print('Repo root:', repo_root)
            print('Added common/lib:', common_lib)
            return repo_root
    raise FileNotFoundError('Could not find repo root (needs lessons/ and common/)')

add_repo_to_path()

In [ ]:
import time, inspect
from fast_sdk.board_sdk import BoardSDK

board = BoardSDK()
print('BoardSDK:', board)
print('Candidates:', [n for n in dir(board) if any(k in n.lower() for k in ('buzzer','rgb','led'))])

if hasattr(board, 'set_rgb'):
    print('set_rgb signature:', inspect.signature(board.set_rgb))
else:
    print('No set_rgb on this BoardSDK')

In [ ]:
# --- Buzzer test ---
def buzzer_try(on=True):
    # try common methods/signatures
    for name in ('set_buzzer', 'buzzer', 'beep'):
        if not hasattr(board, name):
            continue
        fn = getattr(board, name)
        # try int
        try:
            fn(1 if on else 0)
            return True, f'{name}(int)'
        except TypeError:
            pass
        # try bool
        try:
            fn(bool(on))
            return True, f'{name}(bool)'
        except TypeError:
            pass
        # try no-arg (beep)
        try:
            if on:
                fn()
                return True, f'{name}()'
        except Exception:
            pass
    return False, 'no compatible buzzer signature found'

ok, how = buzzer_try(True)
print('Buzzer ON:', ok, how)
time.sleep(0.2)
ok, how = buzzer_try(False)
print('Buzzer OFF:', ok, how)

In [ ]:
# --- RGB test (signature unknown) ---
def try_rgb_call(desc, fn, *args):
    try:
        fn(*args)
        print('OK', desc, 'args=', args)
        return True
    except Exception as e:
        print('NO', desc, 'args=', args, '->', e)
        return False

if not hasattr(board, 'set_rgb'):
    print('No set_rgb available')
else:
    fn = board.set_rgb
    print('Trying common call shapes...')

    # common patterns we try:
    # 1) (r,g,b) 2) ((r,g,b),) 3) (idx,r,g,b) 4) (idx,(r,g,b))
    # If any one works, we run a short red/green/blue cycle.
    patterns = [
        ('set_rgb(r,g,b)', (255,0,0)),
        ('set_rgb((r,g,b))', ((255,0,0),)),
        ('set_rgb(idx,r,g,b)', (0,255,0,0)),
        ('set_rgb(idx,(r,g,b))', (0,(0,255,0))),
    ]

    worked = None
    for desc, args in patterns:
        if try_rgb_call(desc, fn, *args):
            worked = (desc, args)
            break

    if not worked:
        print('No RGB call pattern worked. Paste set_rgb signature output to adjust.')
    else:
        print('Using:', worked[0])
        # If we found a working pattern, run RGB cycle using that shape
        def set_color(r,g,b):
            desc, args0 = worked
            if desc == 'set_rgb(r,g,b)':
                fn(r,g,b)
            elif desc == 'set_rgb((r,g,b))':
                fn((r,g,b))
            elif desc == 'set_rgb(idx,r,g,b)':
                fn(0,r,g,b)
            elif desc == 'set_rgb(idx,(r,g,b))':
                fn(0,(r,g,b))

        print('RGB cycle: red, green, blue, off')
        set_color(255,0,0); time.sleep(0.3)
        set_color(0,255,0); time.sleep(0.3)
        set_color(0,0,255); time.sleep(0.3)
        set_color(0,0,0);   time.sleep(0.2)
        print('Done')